In [5]:
import re
from emoji import demojize
import spacy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
nlp = spacy.load("en_core_web_sm")
df = pd.read_csv('all_data.csv')
df.columns

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# spacy + nltk tokenisation
import spacy

nlp = spacy.load('en_core_web_sm')

Index(['id', 'comment_text', 'split', 'created_date', 'publication_id',
       'parent_id', 'article_id', 'rating', 'funny', 'wow', 'sad', 'likes',
       'disagree', 'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'identity_annotator_count',
       'toxicity_annotator_count'],
      dtype='object')

In [6]:
def clean_text(text, remove_stopwords=False, stem_words=False, lemmatise=False):
    if isinstance(text, str):
        text = demojize(text)
        text = re.sub(r'http\S+', 'URLLINK', text)
        text = re.sub(r'\s+', '', text, flags=re.I)
        # Remove unwanted characters (keep alphanumeric, spaces, and some punctuation)
        text = re.sub(r'[^a-zA-Z0-9\s?!.]', '', text)
        text = text.lower()
        
        
        #TODO replace texts with dict stuff
        
        # Handle contractions
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        
        if remove_stopwords:
            stop_words = nlp.Defaults.stop_words
            text = ' '.join([word for word in text.split() if word not in stop_words])

        if stem_words:
            text = nlp.pipe(text, disable=['parser', 'ner', 'tagger'])
            text = ' '.join([word.lemma_ for word in text])
        
        if lemmatise:
            text = nlp.pipe(text, disable=['parser', 'ner'])
            text = ' '.join([word.lemma_ for word in text])
        
        return text
    else:
        return ""

df['cleaned_comment_text'] = df['comment_text'].apply(clean_text)

def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

tagged_data = [TaggedDocument(words=tokenize(_d), tags=[str(i)]) for i, _d in enumerate(df['cleaned_comment_text'])]

# train the Doc2Vec model
model = Doc2Vec(vector_size=20, min_count=2, epochs=10)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# get the document vectors
document_vectors = [model.infer_vector(tokenize(doc)) for doc in df['cleaned_comment_text']]

[nltk_data] Downloading package punkt to /home/caskei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import pickle 
 
# save tagged_data 
with open('tagged_data.pickle', 'wb') as handle: 
    pickle.dump(document_vectors, handle, protocol=pickle.HIGHEST_PROTOCOL) 